# BETA BANK

## Descripción

Los clientes de Beta Bank se están yendo, cada mes, poco a poco. Los banqueros descubrieron que es más barato salvar a los clientes existentes que atraer nuevos.

Es necesario predecir si un cliente dejará el banco pronto. 

Me han proporcionado los datos sobre el comportamiento pasado de los clientes y la terminación de contratos con el banco.

## Objetivo

Crear un modelo con el máximo valor F1 posible (de al menos 0.59). 

Verificar F1 para el conjunto de prueba. 

Medir la métrica AUC-ROC y compararla con el valor F1.

## Carga de datos y librería

In [991]:
# Cargaré todas las librerías que necesito.
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

In [992]:
# Cargaré los datos del nuevo dataset.
clients = pd.read_csv('/Users/brisna/github_projects/tripleten/beta_bank_sprint_10/Churn.csv')
clients

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


## Preparación de datos

In [993]:
# Revisaré el tipo de datos que tengo y los posibles datos ausentes.
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [994]:
# Veré la descripción de cada columna.
clients.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [995]:
clients.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

Características:

    * RowNumber: índice de cadena de datos
    * CustomerId: identificador de cliente único
    * Surname: apellido
    * CreditScore: valor de crédito
    * Geography: país de residencia
    * Gender: sexo
    * Age: edad
    * Tenure: período durante el cual ha madurado el depósito a plazo fijo de un cliente (años)
    * Balance: saldo de la cuenta
    * NumOfProducts: número de productos bancarios utilizados por el cliente
    * HasCrCard: el cliente tiene una tarjeta de crédito (1 - sí; 0 - no)
    * IsActiveMember: actividad del cliente (1 - sí; 0 - no)
    * EstimatedSalary: salario estimado


Objetivo

    * Exited: El cliente se ha ido (1 - sí; 0 - no)

HALLAZGOS:

* El Dataframe contiene 10,000 y 14 columnas.
  
* Los datos muestran una descripción de algunos clientes, los creditos obtenidos, tiempo de dicho crédito y el saldo de su cuenta. Además revelan información personal del cliente como edad, país, género y su salario estimado.
  
* Observo las etiquetas de las columnas de tipo PascalCase por lo que las cambiaré por snake_case.
  
* Puedo observar que hay datos nulos en 'Tenure'. Los revisaré más adelante.
  
* El valor de crédito promedio es de 650.53 con una desviación estandar de 96.65 lo que indica que la mayor parte de los datos van desde 553.88 hasta los 747.18
  
* La edad promedio de los clientes está entre los 38 - 39 años con una desviación estandar de 10.49 lo que refleja una distrubución moderada.
  
* En promedio, los clientes tienen 5 años en el banco. Puedo observar algunos ceros que quizá se refieran a clientes nuevos.
  
* El promedio del saldo en cuenta es de $76,485.89 con una umbral entre los $14,000 y los $138,000 lo cual es bastante amplio.
  
* El saldo más alto en cuenta es de $250,898.09 y puedo observar que hay clientes sin saldo en cuenta.
  
* El 70.55% de los clientes tienen una tarjeta de crédito.
  
* El 51.51% de los clientes están activos.
  
* El salario estimado promedio es de $100,090.24 con una desviación estandar de $57,510.49 lo que indica una alta variabilidad de los ingresos.
  
* El salario más bajo es de $11.58 y el más alto es de $199,992.48
  
* El 20.37% de los clientes abandonaron el banco.


In [996]:
# Cambiaré las etiquetas de cada columna por un estilo snake_case.
def pascal_to_snake(column_name):
    return re.sub(r'(?<!^)(?=[A-Z])', '_', column_name).lower()

clients.columns = [pascal_to_snake(col) for col in clients.columns]
clients


,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


In [997]:
# Revisaré los datos nulos en 'tenure'.
nulos = clients[clients['tenure'].isna()]
nulos

,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
30,31,15589475,Azikiwe,591,Spain,Female,39,NaN,0.00,3,1,0,140469.38,1
48,49,15766205,Yin,550,Germany,Male,38,NaN,103391.38,1,0,1,90878.13,0
51,52,15768193,Trevisani,585,Germany,Male,36,NaN,146050.97,2,0,0,86424.57,0
53,54,15702298,Parkhill,655,Germany,Male,41,NaN,125561.97,1,0,0,164040.94,1
60,61,15651280,Hunter,742,Germany,Male,35,NaN,136857.00,1,0,0,84509.57,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9944,9945,15703923,Cameron,744,Germany,Male,41,NaN,190409.34,2,1,1,138361.48,0
9956,9957,15707861,Nucci,520,France,Female,46,NaN,85216.61,1,1,0,117369.52,1
9964,9965,15642785,Douglas,479,France,Male,34,NaN,117593.48,2,0,0,113308.29,0
9985,9986,15586914,Nepean,659,France,Male,36,NaN,123841.49,2,1,0,96833.00,0


In [998]:
# Reemplazaré los datos nulos con la mediana.
clients['tenure'] = clients['tenure'].fillna(clients['tenure'].median())
clients.head(5)

,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [999]:
# Revisaré los datos con cero en 'tenure'.
clients[clients['tenure'] == 0]

,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
29,30,15656300,Lucciano,411,France,Male,29,0.0,59697.17,2,1,1,53483.21,0
35,36,15794171,Lombardo,475,France,Female,45,0.0,134264.04,1,1,0,27822.99,1
57,58,15647091,Endrizzi,725,Germany,Male,19,0.0,75888.20,1,0,0,45613.75,0
72,73,15812518,Palermo,657,Spain,Female,37,0.0,163607.18,1,0,1,44203.55,0
127,128,15782688,Piccio,625,Germany,Male,56,0.0,148507.24,1,1,0,46824.08,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9793,9794,15772363,Hilton,772,Germany,Female,42,0.0,101979.16,1,1,0,90928.48,0
9799,9800,15722731,Manna,653,France,Male,46,0.0,119556.10,1,1,0,78250.13,1
9843,9844,15778304,Fan,646,Germany,Male,24,0.0,92398.08,1,1,1,18897.29,0
9868,9869,15587640,Rowntree,718,France,Female,43,0.0,93143.39,1,1,0,167554.86,0


Hay 382 clientes con 'tenure' en cero lo que bien podría indicar que son clientes que aún no cumplen 1 año en el banco.

In [1000]:
# Revisare los duplicados.
clients[clients.duplicated()]

,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited


## Modelos

Entrenaré 3 modelos diferentes para evaluar cuál se comporta mejor al momento de predecir el objetivo.


Eliminaré de mis características las columnas 'exited', 'row_number', 'customer_id' y 'surname'. 

Convertiré mis variables categóricas a numéricas utilizando OHE.

### División de los datos

In [1001]:
# Declararé variables para las características del modelo.
features = clients.drop(['exited', 'row_number', 'customer_id', 'surname'], axis=1)

# Haré lo mismo para el objetivo.
target = clients['exited']
target.value_counts()

exited
0    7963
1    2037
Name: count, dtype: int64

Considerando que el tamaño de mis etiquetas en target esta desbalanceado, en mis modelos agregaré el parametro class_weight='balanced' para solucionar esta situación.

In [1002]:
# Convertiré mis variables categóricas a dummy usando OHE.
features = pd.get_dummies(features, drop_first=True) 
features

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,geography_Germany,geography_Spain,gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,False,False,False
1,608,41,1.0,83807.86,1,0,1,112542.58,False,True,False
2,502,42,8.0,159660.80,3,1,0,113931.57,False,False,False
3,699,39,1.0,0.00,2,0,0,93826.63,False,False,False
4,850,43,2.0,125510.82,1,1,1,79084.10,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5.0,0.00,2,1,0,96270.64,False,False,True
9996,516,35,10.0,57369.61,1,1,1,101699.77,False,False,True
9997,709,36,7.0,0.00,1,0,1,42085.58,False,False,False
9998,772,42,3.0,75075.31,2,1,0,92888.52,True,False,True


Le daré un escalamiento a todas las variables númericas que tenía usando StandardScaler.

In [1003]:
# Estandarizaré las columnas numéricas (credit_score, age, balance, estimated_salary) usando StandardScaler.
columns_scale = ['credit_score', 'age', 'balance', 'estimated_salary']
scaler = StandardScaler()
clients[columns_scale] = scaler.fit_transform(clients[columns_scale])
clients

,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,1,15634602,Hargrave,-0.326221,France,Female,0.293517,2.0,-1.225848,1,1,1,0.021886,1
1,2,15647311,Hill,-0.440036,Spain,Female,0.198164,1.0,0.117350,1,0,1,0.216534,0
2,3,15619304,Onio,-1.536794,France,Female,0.293517,8.0,1.333053,3,1,0,0.240687,1
3,4,15701354,Boni,0.501521,France,Female,0.007457,1.0,-1.225848,2,0,0,-0.108918,0
4,5,15737888,Mitchell,2.063884,Spain,Female,0.388871,2.0,0.785728,1,1,1,-0.365276,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,1.246488,France,Male,0.007457,5.0,-1.225848,2,1,0,-0.066419,0
9996,9997,15569892,Johnstone,-1.391939,France,Male,-0.373958,10.0,-0.306379,1,1,1,0.027988,0
9997,9998,15584532,Liu,0.604988,France,Female,-0.278604,7.0,-1.225848,1,0,1,-1.008643,1
9998,9999,15682355,Sabbatini,1.256835,Germany,Male,0.293517,3.0,-0.022608,2,1,0,-0.125231,1


In [1004]:
# Dividiré los datos para entrenar el modelo.
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, random_state=12345)

features_train, features_valid, target_train, target_valid = train_test_split(features_train, target_train, test_size=0.20, random_state=12345)

In [1005]:
# Aplicaré SMOTE al conjunto de entrenamiento para balancear las clases.
smote = SMOTE(random_state=12345)
features_train_smote, target_train_smote = smote.fit_resample(features_train, target_train)

### Entrenamiento de modelos

#### Random Forest

In [1006]:
# Entrenaré un modelo con Random Forest.
model_random_forest = RandomForestClassifier(class_weight='balanced', random_state=12345)
model_random_forest.fit(features_train_smote, target_train_smote)

# Evaluaré el modelo.
forest_train_pred = model_random_forest.predict(features_train)
forest_valid_pred = model_random_forest.predict(features_valid)
forest_test_pred = model_random_forest.predict(features_test)

In [1007]:
# Calcularé el F1-score.
f1_train_forest = f1_score(target_train, forest_train_pred)
f1_valid_forest = f1_score(target_valid, forest_valid_pred)
f1_test_forest = f1_score(target_test, forest_test_pred)

print(f'F1 score - Entrenamiento: {f1_train_forest}')
print(f'F1 score - Validación: {f1_valid_forest}')
print(f'F1 score - Prueba: {f1_test_forest}')

F1 score - Entrenamiento: 1.0
F1 score - Validación: 0.5945121951219512
F1 score - Prueba: 0.6098901098901099


Observo que el f1_score para entrenamiento es igual a 1 lo que indica un sobreajuste, sobretodo considerando que el fi_score de validación es de 0.54

Intentaré usar GridSearchCV para mejorar el rendimiento del modelo.

In [1008]:
# Usaré GridSearchCV para optimizar los hiperparámetros y maximizar F1-score.
grid = {'n_estimators': list(range(10, 101, 10)), 'max_depth': [10, 20, None], 'min_samples_split': [2, 5], 'min_samples_leaf': [1, 2],}

grid_search_forest = GridSearchCV(estimator=RandomForestClassifier(random_state=12345), param_grid=grid, scoring='f1', cv=5)

grid_search_forest.fit(features_train_smote, target_train_smote)

print(f'Mejor F1 score en validación: {grid_search_forest.best_score_}')
print(f'Mejores parámetros: {grid_search_forest.best_params_}')

Mejor F1 score en validación: 0.8632109210549691
Mejores parámetros: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


In [1009]:
# Evaluaré el modelo con AUC-ROC.
probabilities_test_forest = model_random_forest.predict_proba(features_test)[:, 1]
roc_auc_best_forest = roc_auc_score(target_test, probabilities_test_forest)

print(f'AUC-ROC: {roc_auc_best_forest}')

AUC-ROC: 0.8319792632755464


In [1010]:
# Haré un reporte de clasificación
print(classification_report(target_test, forest_test_pred))

# Compararé F1 y AUC-ROC
print(f"Comparación de métricas: F1 score vs AUC-ROC")
print(f"F1 score del conjunto de prueba: {f1_test_forest}")
print(f"AUC-ROC del conjunto de prueba: {roc_auc_best_forest}")

              precision    recall  f1-score   support

           0       0.90      0.89      0.89      1965
           1       0.60      0.62      0.61       535

    accuracy                           0.83      2500
   macro avg       0.75      0.75      0.75      2500
weighted avg       0.83      0.83      0.83      2500

Comparación de métricas: F1 score vs AUC-ROC
F1 score del conjunto de prueba: 0.6098901098901099
AUC-ROC del conjunto de prueba: 0.8319792632755464


#### Decision Tree Classifier 

In [1011]:
# Entrenaré un modelo con Decision Tree Classifier.
model_tree = DecisionTreeClassifier(class_weight='balanced', random_state=12345)
model_tree.fit(features_train_smote, target_train_smote)

# Evaluaré el modelo.
tree_train_pred = model.predict(features_train)
tree_valid_pred = model.predict(features_valid)
tree_test_pred = model.predict(features_test)

In [1012]:
# Calcularé el F1-score.
f1_train_tree = f1_score(target_train, tree_train_pred)
f1_valid_tree = f1_score(target_valid, tree_valid_pred)
f1_test_tree = f1_score(target_test, tree_test_pred)

print(f'F1 score - Entrenamiento: {f1_train_tree}')
print(f'F1 score - Validación: {f1_valid_tree}')
print(f'F1 score - Prueba: {f1_test_tree}')

F1 score - Entrenamiento: 1.0
F1 score - Validación: 0.5484536082474227
F1 score - Prueba: 0.5527272727272727


In [1013]:
# Configuraré GridSearchCV para optimizar los hiperparámetros.
grid_tree = {'max_depth': [3, 5, 7, 10, 15], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}

grid_search_tree = GridSearchCV(estimator=model_tree, param_grid=grid_tree, scoring='f1', cv=5, n_jobs=-1, verbose=1)

grid_search_tree.fit(features_train_smote, target_train_smote)

print("Mejores parámetros encontrados:", grid_search_tree.best_params_)
print("Mejor F1 score en validación cruzada:", grid_search_tree.best_score_)

Fitting 5 folds for each of 45 candidates, totalling 225 fits


Mejores parámetros encontrados: {'max_depth': 7, 'min_samples_leaf': 1, 'min_samples_split': 2}
Mejor F1 score en validación cruzada: 0.8131183248974416


In [1014]:
best_model_tree = grid_search_tree.best_estimator_
best_model_tree.fit(features_train_smote, target_train_smote)

# Predicciones y métricas en el conjunto de validación
valid_pred_best_tree = best_model_tree.predict(features_valid)
f1_valid_best_tree = f1_score(target_valid, valid_pred_best_tree)
print("F1 Score (Decision Tree) en Validación:", f1_valid_best_tree)

# Predicciones y métricas en el conjunto de prueba
test_pred_best_tree = best_model_tree.predict(features_test)
f1_test_best_tree = f1_score(target_test, test_pred_best_tree)

F1 Score (Decision Tree) en Validación: 0.5948275862068966


In [1015]:
# Evaluaré el modelo con AUC-ROC.
probabilities_test_best_tree = best_model_tree.predict_proba(features_test)[:, 1]
roc_auc_best_tree = roc_auc_score(target_test, probabilities_test_best_tree)

print("F1 Score (Decision Tree) en Prueba:", f1_test_best_tree)
print("AUC-ROC (Decision Tree):", roc_auc_best_tree)

F1 Score (Decision Tree) en Prueba: 0.5975395430579965
AUC-ROC (Decision Tree): 0.814214644122613


In [1016]:
# Haré un reporte de clasificación.
print(classification_report(target_test, test_pred_best_tree))

# Compararé F1 y AUC-ROC
print(f"Comparación de métricas: F1 score vs AUC-ROC")
print(f"F1 score del conjunto de prueba: {f1_test_best_tree}")
print(f"AUC-ROC del conjunto de prueba: {roc_auc_best_tree}")

              precision    recall  f1-score   support

           0       0.90      0.87      0.88      1965
           1       0.56      0.64      0.60       535

    accuracy                           0.82      2500
   macro avg       0.73      0.75      0.74      2500
weighted avg       0.83      0.82      0.82      2500

Comparación de métricas: F1 score vs AUC-ROC
F1 score del conjunto de prueba: 0.5975395430579965
AUC-ROC del conjunto de prueba: 0.814214644122613


#### Logistic Regression

In [1017]:
# Entrenaré un modelo con Logistic Regression.
model_logreg = LogisticRegression(random_state=12345, class_weight='balanced', solver='liblinear')
model_logreg.fit(features_train_smote, target_train_smote)

# Evaluaré el modelo.
train_pred_logreg = model_logreg.predict(features_train)
valid_pred_logreg = model_logreg.predict(features_valid)
test_pred_logreg = model_logreg.predict(features_test)

In [1018]:
# Calcularé el F1-score.
f1_train_logreg = f1_score(target_train, train_pred_logreg)
f1_valid_logreg = f1_score(target_valid, valid_pred_logreg)
f1_test_logreg = f1_score(target_test, test_pred_logreg)

# Imprimir los valores F1
print(f'F1 score (Logistic Regression) - Entrenamiento: {f1_train_logreg}')
print(f'F1 score (Logistic Regression) - Validación: {f1_valid_logreg}')
print(f'F1 score (Logistic Regression) - Prueba: {f1_test_logreg}')

F1 score (Logistic Regression) - Entrenamiento: 0.43887665198237885
F1 score (Logistic Regression) - Validación: 0.4553846153846154
F1 score (Logistic Regression) - Prueba: 0.45460160592958615


In [1019]:
# Evaluaré el modelo con AUC-ROC.
probabilities_test_logreg = model_logreg.predict_proba(features_test)[:, 1]
roc_auc_logreg = roc_auc_score(target_test, probabilities_test_logreg)

print(f'AUC-ROC (Logistic Regression): {roc_auc_logreg}')

AUC-ROC (Logistic Regression): 0.7141784975387029


In [1020]:
# Haré un reporte de clasificación.
print(classification_report(target_test, test_pred_logreg))

# Compararé F1 y AUC-ROC
print(f"Comparación de métricas: F1 score vs AUC-ROC")
print(f"F1 score del conjunto de prueba: {f1_test_logreg}")
print(f"AUC-ROC del conjunto de prueba: {roc_auc_logreg}")

              precision    recall  f1-score   support

           0       0.88      0.64      0.74      1965
           1       0.34      0.69      0.45       535

    accuracy                           0.65      2500
   macro avg       0.61      0.66      0.60      2500
weighted avg       0.77      0.65      0.68      2500

Comparación de métricas: F1 score vs AUC-ROC
F1 score del conjunto de prueba: 0.45460160592958615
AUC-ROC del conjunto de prueba: 0.7141784975387029


### Comparación de modelos

In [1021]:
# Reporte de clasificación para Decision Tree
print("\n--- Decision Tree ---")
print(classification_report(target_test, test_pred_best_tree))

# Reporte de clasificación para Random Forest
print("\n--- Random Forest ---")
print(classification_report(target_test, forest_test_pred))

# Reporte de clasificación para Logistic Regression
print("\n--- Logistic Regression ---")
print(classification_report(target_test, test_pred_logreg))

# Comparación de métricas
print("\n--- Comparación de métricas ---")
print(f"F1 score (Decision Tree) - Prueba: {f1_test_tree}")
print(f"AUC-ROC (Decision Tree): {roc_auc_best_tree}")

print(f"F1 score (Random Forest) - Prueba: {f1_test_best_tree}")
print(f"AUC-ROC (Random Forest): {roc_auc_best_forest}")

print(f"F1 score (Logistic Regression) - Prueba: {f1_test_logreg}")
print(f"AUC-ROC (Logistic Regression): {roc_auc_logreg}")


--- Decision Tree ---
              precision    recall  f1-score   support

           0       0.90      0.87      0.88      1965
           1       0.56      0.64      0.60       535

    accuracy                           0.82      2500
   macro avg       0.73      0.75      0.74      2500
weighted avg       0.83      0.82      0.82      2500


--- Random Forest ---
              precision    recall  f1-score   support

           0       0.90      0.89      0.89      1965
           1       0.60      0.62      0.61       535

    accuracy                           0.83      2500
   macro avg       0.75      0.75      0.75      2500
weighted avg       0.83      0.83      0.83      2500


--- Logistic Regression ---


              precision    recall  f1-score   support

           0       0.88      0.64      0.74      1965
           1       0.34      0.69      0.45       535

    accuracy                           0.65      2500
   macro avg       0.61      0.66      0.60      2500
weighted avg       0.77      0.65      0.68      2500


--- Comparación de métricas ---
F1 score (Decision Tree) - Prueba: 0.5527272727272727
AUC-ROC (Decision Tree): 0.814214644122613
F1 score (Random Forest) - Prueba: 0.5975395430579965
AUC-ROC (Random Forest): 0.8319792632755464
F1 score (Logistic Regression) - Prueba: 0.45460160592958615
AUC-ROC (Logistic Regression): 0.7141784975387029


### Conclusiones finales

Con base en los resultados obtenidos, puedo observar que: 

* Random Forest muestra el mejor F1-score en la clase 1, alcanzando 0.61, lo que ayuda a identificar clientes que probablemente saldrán. También tiene un AUC-ROC de 0.83, que indica una buena capacidad para distinguir entre clientes que permanecerán y aquellos que se irán.

* Decision Tree tiene un F1-score de 0.60 y un AUC-ROC de 0.81. Aunque tiene un buen rendimiento, el modelo de Random Forest tiene mayor precisión en la clasificación.

* Logistic Regression obtuvo un F1-score de 0.45 y un AUC-ROC de 0.71, lo cual es significativamente menor que el modelo de Random Forest y de Decision Tree.

Por lo tanto, recomendaría el modelo Random Forest para Beta Bank, considerando el equilibrio que hay entre F1-score y AUC-ROC en comparación con los otros modelos.